In [1]:
!curl http://data.csail.mit.edu/im2recipe/recipe1M_layers.tar.gz --output recipe1M_layers.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  380M  100  380M    0     0   981k      0  0:06:37  0:06:37 --:--:-- 1010k  9 36.7M    0     0  1112k      0  0:05:50  0:00:33  0:05:17 1045k  0:06:24  0:02:04  0:04:20  967k   0  1012k      0  0:06:24  0:02:09  0:04:15  969k11k      0  0:06:25  0:02:11  0:04:14  978k  999k      0  0:06:29  0:02:21  0:04:08  748k    0   991k      0  0:06:33  0:02:31  0:04:02  920k0   986k      0  0:06:34  0:03:15  0:03:19  937k   0   983k      0  0:06:36  0:04:18  0:02:18  985k  0  0:06:36  0:05:55  0:00:41  989k


In [2]:
!pip install tqdm
!tar -xzvf recipe1M_layers.tar.gz
!rm recipe1M_layers.tar.gz

x layer1.json
x layer2.json


In [13]:
!unzip recipe-ingredients-dataset.zip -d recipe-ingr/
!rm recipe-ingredients-dataset.zip

Archive:  recipe-ingredients-dataset.zip
  inflating: recipe-ingr/test.json   
  inflating: recipe-ingr/train.json  


In [3]:
import json
from pprint import pprint
from string import ascii_lowercase
import random
from tqdm import tqdm

clean_ingredients = set()
all_recipes = set()

### Recipe Ingredient

In [14]:
with open('recipe-ingr/train.json', 'rb') as f:
    recipe_ingr_train = json.loads(f.read().decode("utf-8"))
with open('recipe-ingr/test.json', 'rb') as f:
    recipe_ingr_test = json.loads(f.read().decode("utf-8"))

recipe_ingr = recipe_ingr_train + recipe_ingr_test
pprint(recipe_ingr[0])

{'cuisine': 'greek',
 'id': 10259,
 'ingredients': ['romaine lettuce',
                 'black olives',
                 'grape tomatoes',
                 'garlic',
                 'pepper',
                 'purple onion',
                 'seasoning',
                 'garbanzo beans',
                 'feta cheese crumbles']}


In [16]:
for r in recipe_ingr:
    clean_ingredients.update(r['ingredients'])
recipe_ingr_recipes = {tuple(sorted([x.lower() for x in r['ingredients']])) for r in recipe_ingr}

### Recipe1M

In [6]:
with open('layer1.json', 'r') as f:
    recipe1M_data = json.loads(f.read())

In [7]:
recipe1M_recipes = []
recipe1M_names = []
recipe1M_id = []
for r in recipe1M_data:
    recipe1M_recipes.append(tuple(sorted([x['text'].lower() for x in r['ingredients']])))
    recipe1M_names.append(r['title'])
    recipe1M_id.append(r['id'])

In [12]:
recipe1M_recipes[0]

('(makes about 4 cups)',
 '1 ounce cheddar, grated (1/4 cup)',
 '1 ounce gruyere cheese, grated (1/4 cup)',
 '1/2 teaspoon kosher salt',
 '1/3 cup all-purpose flour',
 '1/4 cup (1/2 stick) unsalted butter',
 '1/4 to 1/2 teaspoon chipotle chili powder',
 '1/4 to 1/2 teaspoon chipotle chili powder (see note)',
 '1/8 teaspoon garlic powder',
 '14 ounces semihard cheese (page 23), grated (about 3 1/2 cups)',
 '2 cups beechers flagship cheese sauce (recipe follows)',
 '2 ounces semisoft cheese (page 23), grated (1/2 cup)',
 '3 cups milk',
 '6 ounces penne')

In [11]:
recipe1M_names[0]

'Worlds Best Mac and Cheese'

### Clean

In [17]:
def replace_units(s):
    original_string = s
    
    tokens = s.replace('to taste', '').split()
    units = ['ounce', 'ounces', 'cups', 'cup', 'teaspoon', 'tablespoon', 'tablespoons', 'teaspoons', 'c', 'g', 'v', 'tbsp', 'x', 'ml', 'lb', 'tbs', 'oz', 'pkg', 'large', 'small', 'tsp', 'inch', 'grams', 'quarts', 'lbs', 'can', 'cube', 'whole', 'or', 'pieces', 'piece', 'chopped', 'shredded', 'diced', 'fresh', 'crushed']
    tokens_new = []
    for t in tokens:
        if t.strip() not in units:
            tokens_new.append(t)
            
    #if original_string != ' '.join(tokens_new):
    #    print(original_string, '--->', ' '.join(tokens_new))
    
    return ' '.join(tokens_new)
    
def cleanup_ingredient_list(l, remove_whitespace_variants):
    l = {replace_units(''.join([char for char in x.lower().strip() if char in ascii_lowercase + ' ']).strip()) for x in l}
    if not remove_whitespace_variants:
        return l
    
    removal_mapping = dict()
    remove_ingr = set()
    for i in l:
        without_whitespace = ''.join([char for char in i if char in ascii_lowercase])
        if without_whitespace in l and i != without_whitespace:
            remove_ingr.add(without_whitespace)
            removal_mapping[without_whitespace] = i
    l -= remove_ingr
    #print('removed: ' + str(remove_ingr))
    return l, removal_mapping

In [18]:
messy_recipes = recipe1M_recipes
clean_recipes = recipe_ingr_recipes

In [19]:
clean_ingredients, whitespace_mapping = cleanup_ingredient_list(clean_ingredients, remove_whitespace_variants=True)

new_messy_recipes = []
for r in tqdm(messy_recipes):
    new_messy_recipes.append(tuple(cleanup_ingredient_list(r, remove_whitespace_variants=False)))
messy_recipes = new_messy_recipes

100%|██████████| 1029720/1029720 [02:37<00:00, 6549.52it/s]


In [34]:
messy_recipes[1]

('sliced celery',
 'vinegar',
 'minced green pepper',
 'mayonnaise possibly salad dressing',
 'cubed american cheese',
 'dry dill weed',
 'salt',
 'elbow macaroni',
 'minced pimento')

In [35]:
recipe1M_recipes

1029720

In [27]:
whitespace_mapping

{'gingerroot': 'ginger root',
 'blackpepper': 'black pepper',
 'mahimahi': 'mahi mahi',
 'dillweed': 'dill weed',
 'cornflour': 'corn flour',
 'codfish': 'cod fish',
 'poppyseeds': 'poppy seeds',
 'peapods': 'pea pods',
 'lemongrass': 'lemon grass',
 'cornflakes': 'corn flakes',
 'arrowroot': 'arrow root',
 'wheatberries': 'wheat berries',
 'ladyfingers': 'lady fingers',
 'cuminseed': 'cumin seed',
 'piecrust': 'pie crust',
 'poundcake': 'pound cake',
 'sugarcane': 'sugar cane',
 'tatsoi': 'tat soi',
 'shiromiso': 'shiro miso',
 'rosewater': 'rose water',
 'crabmeat': 'crab meat',
 'cornbread': 'corn bread'}

In [28]:
new_messy_recipes = []
for r in messy_recipes:
    r_new = set()
    for i in r:
        if i in whitespace_mapping:
            r_new.add(whitespace_mapping[i])
        else:
            r_new.add(i)
    new_messy_recipes.append(tuple(r_new))
messy_recipes = new_messy_recipes

new_clean_recipes = []
for r in clean_recipes:
    r_new = set()
    for i in r:
        if i in whitespace_mapping:
            r_new.add(whitespace_mapping[i])
        else:
            r_new.add(i)
    new_clean_recipes.append(tuple(r_new))
clean_recipes = new_clean_recipes

### Save

In [33]:
import pickle
with open('data.pickle', 'wb+') as f:
    pickle.dump({'clean_recipes': clean_recipes, 'messy_recipes': messy_recipes, 'messyname':recipe1M_names, 'messyid':recipe1M_id}, f)